# [LEGALST-123] Lab 05: Central Limit Theorem, Confidence Intervals, Hypothesis Testing

In [1]:
from datascience import *
from collections import Counter
import numpy as np
import pandas as pd
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import plotly.express as px

## Introduction
In this lab, we aim to prepare students for prediction exercises in PSET 1 and PSET 2 by allowing students to contextualize the statistical ideas of the Central Limit Theorem and hypothesis testing by using a dataset containing continuous variables. 

<br/>

<hr style="border: 1px solid #fdb515;" />

## Data & Exploratory Data Analysis

For this lab, we'll be using the same datasets used in our previous labs: the Nashville police stops dataset. Run the following cell below to read the `DataFrame`.

In [2]:
stops = pd.read_csv("https://github.com/ds-modules/data/raw/main/nashville_sample.csv", index_col=0)
stops.head()

,index,raw_row_number,date,time,location,lat,lng,precinct,reporting_area,zone,...,raw_traffic_citation_issued,raw_misd_state_citation_issued,raw_suspect_ethnicity,raw_driver_searched,raw_passenger_searched,raw_search_consent,raw_search_arrest,raw_search_warrant,raw_search_inventory,raw_search_plain_view
0,1840907,93347,2010-04-18,13140.0,"BURGESS AVE & WHITE BRIDGE PIKE, NASHVILLE, TN...",36.145004,-86.857970,1.0,5103.0,113.0,...,False,NaN,N,False,False,False,False,False,False,False
1,492044,2001428,2015-01-19,19920.0,"DUE WEST AVE W & S GRAYCROFT AVE, MADISON, TN,...",36.249187,-86.734459,7.0,1797.0,723.0,...,False,False,N,False,False,False,False,False,False,False
2,431170,1996331,2015-01-15,1020.0,"S GALLATIN PIKE & MADISON BLVD, MADISON, TN, 3...",36.254979,-86.715246,7.0,1623.0,711.0,...,False,False,N,False,False,False,False,False,False,False
3,2066423,1319451,2013-05-17,62760.0,"CHARLOTTE PIKE & W HILLWOOD DR, NASHVILLE, TN,...",36.139093,-86.880533,1.0,5009.0,123.0,...,False,False,N,False,False,False,False,False,False,False
4,2899480,201349,2010-09-01,28140.0,"BELL RD & DODSON CHAPEL RD, HERMITAGE, TN, 37076",36.163310,-86.613147,5.0,9501.0,521.0,...,False,NaN,N,False,False,False,False,False,False,False


Let's refer back to our last lab, where we explored different distributions using histograms. In particular, we looked at the distribution of stop counts for `"subject_sex"` and `"subject_age"`. For this notebook, let's look at the distribution of **age for each race**.

<!-- BEGIN QUESTION -->
<div class=“alert alert-warning”>

#### **Question 1.1**:
Before explore these variables, let's clean the dataset. In the code cell below, drop any columns that have "raw" in their column names. Then, drop the rows with *any* null values EXCEPT for the columns` "contraband_found"`, `"contraband_drugs"`, `"contraband_weapons"`, `"search_basis"`, and `"notes"`.
</div>

Hint: Look at Lab 04 question 1.1 and question 1.2! It should be very similar. 

In [3]:
# YOUR ANSWER HERE. You can use more or less lines than provided below. 
# ... 
# ... 
# ...
# stops = ...

# stops.head(5)

In [4]:
# SOLUTION 
raw_columns = [col for col in stops.columns if 'raw_' in col]
necessary_cols = stops.columns.drop(["contraband_found", "contraband_drugs", "contraband_weapons", "search_basis", "notes"]) #SOLUTION 
stops = stops.dropna(subset=necessary_cols,axis=0)
stops = stops.drop(raw_columns, axis=1)

stops.head(5)

,index,date,time,location,lat,lng,precinct,reporting_area,zone,subject_age,...,contraband_drugs,contraband_weapons,frisk_performed,search_conducted,search_person,search_vehicle,search_basis,reason_for_stop,vehicle_registration_state,notes
1,492044,2015-01-19,19920.0,"DUE WEST AVE W & S GRAYCROFT AVE, MADISON, TN,...",36.249187,-86.734459,7.0,1797.0,723.0,45.0,...,NaN,NaN,False,False,False,False,NaN,vehicle equipment violation,TN,tail light out
2,431170,2015-01-15,1020.0,"S GALLATIN PIKE & MADISON BLVD, MADISON, TN, 3...",36.254979,-86.715246,7.0,1623.0,711.0,21.0,...,NaN,NaN,False,False,False,False,NaN,moving traffic violation,TN,NaN
3,2066423,2013-05-17,62760.0,"CHARLOTTE PIKE & W HILLWOOD DR, NASHVILLE, TN,...",36.139093,-86.880533,1.0,5009.0,123.0,35.0,...,NaN,NaN,False,False,False,False,NaN,vehicle equipment violation,TN,NaN
5,543402,2012-12-13,67560.0,"3RD AVE S & ASH ST, NASHVILLE, TN, 37210",36.152786,-86.770562,4.0,4005.0,425.0,26.0,...,NaN,NaN,False,False,False,False,NaN,moving traffic violation,TN,RAN STOP SIGN
6,1446083,2012-02-26,55380.0,"BELL RD & DODSON CHAPEL RD, HERMITAGE, TN, 37076",36.163310,-86.613147,5.0,9501.0,521.0,18.0,...,NaN,NaN,False,False,False,False,NaN,moving traffic violation,TN,NaN


Let's explore the columns `"subject_race"` and `"subject_age"`. For convenience, we have provided code below showing what race/ethincity categories exist within the column  `"subject_race"`. For this particular question, let's look at the distribution of people who are categorized as `'hispanic'` or `'white'` for `"subject_race"`. 

In [5]:
stops["subject_race"].unique()

array(['black', 'white', 'hispanic', 'unknown', 'asian/pacific islander',
       'other'], dtype=object)

<!-- BEGIN QUESTION -->
<div class=“alert alert-warning”>

#### **Question 1.2**:

In the cell below, use Plotly to plot a histogram showing the distribution of age for people who are categorized as Hispanic with the y axis representing percentage. To do this, first create a table called `subject_hispanic` with your manipulations. Use `range_x=[0,80]` to properly scale the histogram.

Hint 1: [here](https://plotly.github.io/plotly.py-docs/generated/plotly.express.histogram.html) is the documentation for Plotly's histogram method.

Hint 2: Take a look at the `histnorm` attribute of the histogram method!

</div>

In [6]:
# YOUR CODE HERE

# subject_hispanic = ... 

# Histogram

In [7]:
# SOLUTION
subject_hispanic = stops[stops['subject_race'] == 'hispanic']
px.histogram(subject_hispanic, x='subject_age', histnorm='percent', range_x=[0, 80])

**Find the 25th, 50th, 75th percentile of `subject_hispanic` below.**

In [8]:
# YOUR CODE HERE
# Find the 25th percentile

In [9]:
# Find the 50th percentile

In [10]:
# Find the 75th percentile

In [11]:
# SOLUTIONS
percentile(25, subject_hispanic['subject_age'])
percentile(50, subject_hispanic['subject_age'])
percentile(75, subject_hispanic['subject_age'])

33.0

<!-- BEGIN QUESTION -->
<div class=“alert alert-warning”>

#### **Question 1.3**:

Now, follow the same process for people who are categorized as White. Again, to do this, first create a table called `subject_white` with your manipulations.

</div>

In [12]:
# YOUR CODE HERE

# subject_white = ... 

# Histogram

In [13]:
# SOLUTION
subject_white = stops[stops['subject_race'] == 'white']
px.histogram(subject_white, x='subject_age', histnorm='percent', range_x=[0, 80])

**Find the 25th, 50th, 75th percentile of `subject_white` below.**

In [ ]:
# YOUR CODE HERE
# Find the 25th percentile

In [ ]:
# Find the 50th percentile

In [ ]:
# Find the 75th percentile

In [ ]:
# SOLUTIONS
percentile(25, subject_white['subject_age'])
percentile(50, subject_white['subject_age'])
percentile(75, subject_white['subject_age'])

37.0

<!-- BEGIN QUESTION -->
<div class=“alert alert-warning”>

#### **Question 1.4**:

Now, create an overlaid historgram comparing the two distributions.

</div>

In [ ]:
# YOUR ANSWER HERE

In [14]:
# SOLUTION
combined = pd.concat([subject_hispanic, subject_white])
px.histogram(combined, x='subject_age', color='subject_race', histnorm='percent',barmode='overlay')

**What do you notice about the two distributions? How do they compare?**

_YOUR ANSWER HERE_

## Bootstrapping and the Confidence Interval

Bootstrapping is a statistical technique that allows us to make educated guesses about a population using only a small sample from that population. It works by repeatedly taking small random samples from the data we have and then using these samples to estimate things like averages, variances, or other statistics, as if we had data for the entire population. This technique helps us understand how uncertain or variable our estimates are and is especially useful when we don't have access to the whole population's data.


###  A Random Sample and an Estimate
Let's first draw from the sample, at random with replacement, the same number of times as the original sample size.

It is important to resample the same number of times as the original sample size. The reason is that the variability of an estimate depends on the size of the sample.

If we drew  at random without replacement, we would just get the same sample back. By drawing with replacement, we create the possibility for the new samples to be different from the original, because some participants might be drawn more than once and others not at all.

In [115]:
# Run this cell a few times to see how the distribution varies
h_resampled = stops[stops['subject_race'] == 'hispanic'].sample(len(stops['subject_race'] == 'hispanic'), replace = True)

title = 'Bootstrapped Distribution of Age among the Hispanic Population in our Dataset'
px.histogram(h_resampled, x='subject_age', histnorm='percent', range_x=[0, 80], title=title)

### Resampling from the Sample

By resampling again and again, we can get many such estimates, and hence an `empirical distribution` of the estimates.

Let us collect this code and define a function one_bootstrap_mean that returns one bootstrapped mean of `subject_age`, based on bootstrapping our original dataset.

In [102]:
def one_bootstrap_mean():
    resampled =  stops[stops['subject_race'] == 'hispanic'].sample(len(stops['subject_race'] == 'hispanic'), replace = True)
    bootstrapped_mean = np.mean(resampled['subject_age'])
    return bootstrapped_mean

Run the cell below a few times to see how the bootstrapped means vary. Remember that each of them is an estimate of the population mean.

In [116]:
one_bootstrap_mean()

28.654396728016359

We can now repeat the bootstrap process multiple times by running a `for` loop as usual. In each iteration, we will call the function `one_bootstrap_mean` to generate one value of the bootstrapped mean based on our original dataset. Then we will append the boostrapped mean to the collection array `bstrap_means`.

Let's do 2000 repetitions for this round of bootstrapping! (since this is a large number the code might take a while to run)

In [118]:
num_repetitions = 10000
bstrap_means = make_array()
for i in np.arange(num_repetitions):
    bstrap_means = np.append(bstrap_means, one_bootstrap_mean())

Now let's visualize what we got from the bootstrapped process

In [134]:
title = 'Distribution of Bootstrapped Mean Age Amongst Hispanic Population'
px.histogram(bstrap_means,histnorm='percent', title=title)


### Confidence Interval

Confidence intervals are an important tool in data science. They help us create an estimate of a population parameter from a subset of the data. Here, we are using a 95% confidence interval to guess the average age of the Hispanic population that has been stopped by police.

In [135]:
# Get the endpoints of the 95% confidence interval
left = percentile(2.5, bstrap_means)
right = percentile(97.5, bstrap_means)

make_array(left, right)

array([ 27.94887526,  29.27607362])

Now let's add the confidence interval to the histogram above

In [152]:
title = 'Distribution of Bootstrapped Mean Age Amongst Hispanic Population'
fig = px.histogram(bstrap_means,histnorm='percent', title=title)
fig.add_shape(type='line', x0=left, y0=0, x1=right, y1=0, line_color='red')

Here, you can visualize our estimate for the population age as displayed by the red line.

## 4. Central Limit Theorem (CLT)

**The Central Limit Theorem (CLT)** is a fundamental concept in statistics that has significant implications for making inferences about populations based on samples. It states that, regardless of the shape of the original population distribution, the distribution of the sample means will approach a normal distribution as the sample size increases. This is true as long as the sample size is sufficiently large.

The significance of the Central Limit Theorem lies in its ability to provide a bridge between the characteristics of a population and the properties of the sample means drawn from that population.

**Population Mean:**

The Central Limit Theorem tells us that the sampling distribution of the mean of a random sample will be approximately normally distributed, even if the population distribution is not normal.
This is crucial because it allows us to make inferences about the population mean using statistical methods that assume a normal distribution.

**Sample Size:**

The larger the sample size, the closer the distribution of the sample mean will be to a normal distribution according to the CLT.
As the sample size increases, the standard deviation of the sampling distribution decreases. This means that larger sample sizes provide more precise estimates of the population mean.

To demonstrate this, let's look at the original dataset and data we obtained from bootstrapping above.

In [156]:
title = 'Bootstrapped Distribution of Age among the Hispanic Population in our Dataset'
px.histogram(h_resampled, x='subject_age', histnorm='percent', range_x=[0, 80], title=title)

Now, let's look at the distribution of bootstrapped mean amongst this sample

In [157]:
title = 'Distribution of Bootstrapped Mean Age Amongst Hispanic Population'
px.histogram(bstrap_means,histnorm='percent', title=title)

As we can see from above, the original distribution looks nothing like a normal distribution, but the bootstrapped means still follow a somewhat normal shape.

Now, let's connect the Central Limit Theorem to the motivation behind using **regression**:

In regression analysis, the Central Limit Theorem is often invoked when dealing with the distribution of the regression coefficients.
The ordinary least squares (OLS) estimators, which are commonly used in regression analysis, are unbiased and efficient under the assumption of normally distributed errors.
The CLT justifies the use of statistical tests and confidence intervals for regression coefficients, as it ensures that the distribution of these coefficients becomes approximately normal as the sample size increases.

In summary, the Central Limit Theorem is significant because it allows statisticians to make valid inferences about population parameters, particularly the population mean, based on samples. This is crucial in various fields, including regression analysis, where assumptions about the distribution of coefficients play a key role in drawing conclusions about relationships between variables.

## Here is an AB test that I created, the original test here was incorrect (ie. held no real statistical basis). I did not have time to create the cells for it but this is a good starting point.

Here, we are testing whether the underlying distribution of age is the same amongst Hispanic and White populations using a 5% p-value cutoff. The test statistic being used is the absolute difference between the average ages.

In [ ]:

ab_dataframe = combined[['subject_age', 'subject_race']]
test_stat = np.abs(np.mean(ab_dataframe[ab_dataframe['subject_race'] == 'hispanic']['subject_age']) - np.mean(ab_dataframe[ab_dataframe['subject_race'] == 'white']['subject_age']))

simulated_stats = np.array([])

for i in range(10000):
    new_column = ab_dataframe.sample(ab_dataframe.shape[0], replace=False).reset_index()['subject_race']

    sampled = ab_dataframe.copy()
    sampled.loc[:, 'shuffled'] = new_column

    new_stat = np.abs(np.mean(sampled[sampled['shuffled'] == 'hispanic']['subject_age']) - np.mean(sampled[sampled['shuffled'] == 'white']['subject_age']))
    simulated_stats = np.append(simulated_stats, new_stat)

In [ ]:
print('Our observed test statistic is', test_stat)

Our observed test statistic is 10.4041666667


In [ ]:
px.histogram(simulated_stats)

In [ ]:
p_val = np.count_nonzero(simulated_stats >= test_stat) / len(simulated_stats)
print('our p value is', p_val)

our p value is 0.0814


since this value is greater than our 0.05 cutoff, we fail to reject the null hypothesis that the distributions are different.